# The Variance Estimator
Let $P_\mathbf{x} := \mathbb{P}(\mathbf{z} = \mathbf{x})$.
Let $\mathbf{z}^*$ be the realized treatment vector.
Let $P^* := \mathbb{P}(\mathbf{z} = \mathbf{z^*})$ and $P_S^* := \mathbb{P}(\mathbf{z_S} = \mathbf{z_S^*})$ for $S\subseteq[n]$.
Then, the variance estimator is given by $\widehat{\text{Var}}(\widehat{TTE}) = (*) + (**)$ where 
$$(*) = \frac{1}{n^2} \sum_{i=1}^n \sum_{j \in M_i} \frac{1}{P_{N_i \cup N_j}^*} \cdot Y_i(\mathbf{z}^*)w_i(\mathbf{z}^*) \cdot Y_j(\mathbf{z}^*)w_j(\mathbf{z}^*)\cdot \text{Cov}_{ij}$$
and
$$(**) = \frac{1}{n^2} \sum_{i=1}^n P_{N_i}^* \cdot Y_i^2\mathbf{z^*} w_i^2\mathbf{z^*} \cdot \sum_{j \in M_i} 2^{|N_j|} - 2^{|N_j\setminus N_i|}$$
with 
$$\text{Cov}_{ij} := \prod_{k \in N_i \cup N_j} p^{\mathbf{z}_k^*}(1-p)^{1-\mathbf{z}_k^*}\left(1 - \prod_{\ell \in N_i \cap N_j} p^{\mathbf{z}_\ell^*}(1-p)^{1-\mathbf{z}_\ell^*}\right).$$

## Computing Term 1
We compute the first term, $(*)$, as follows:
- First, create an $n \times 1$ array $\mathbf{YW}$ with entry $i$ equal to $Y_i(\mathbf{z}^*)w_i(\mathbf{z}^*)$.
- Next, we create an $n \times 1$ array $\mathbf{V}$ with entry $i$ equal to $$\sum_{j \in M_i} \frac{1}{P_{N_i \cup N_j}^*} \cdot Y_j(\mathbf{z}^*)w_j(\mathbf{z}^*)\cdot \text{Cov}_{ij}.$$
    -- For each $i \in [n]$, we create the following $m_i := |M_i| \times 1$ arrays: 
    $$\mathbf{P}_i := \begin{bmatrix} 1/P_{N_i \cup N_{j_1}} \\ \vdots \\ 1/P_{N_i \cup N_{j_{m_i}}} \end{bmatrix},\ 
    \mathbf{YW}_i = \begin{bmatrix} Y_{j_1}(\mathbf{z}^*)w_{j_1}(\mathbf{z}^*) \\ \vdots \\ Y_{j_{m_i}}(\mathbf{z}^*)w_{j_{m_i}}(\mathbf{z}^*) \end{bmatrix}, \
    \textbf{COV}_i := \begin{bmatrix} \text{Cov}_{i{j_1}} \\ \vdots \\ \text{Cov}_{i{j_{m_i}}} \end{bmatrix}.$$

    -- Then, entry-wise multiply these arrays to get one array, sum the entries of the resulting array, and that sum is the $i$-th entry of $\mathbf{V}$.
- Then, return the dot product of $\mathbf{YW}$ and $\mathbf{V}$ divided by $n^2$.

## Computing Term 2
We compute the second term, $(**)$, as follows:
- First, compute an $n \times 1$ array $\mathbf{PY2W2}$ where entry $i$ equals $P_{N_i}^* Y_i^2(\mathbf{z}^*) w_i^2(\mathbf{z}^*).$
- Then, compute an $n \times 1$ array $\mathbf{CNTS}$ where entry $i$ equals $$\sum_{j\in M_i} 2^{|N_j|} - 2^{|N_j \setminus N_i|}.$$
- Finally, take the dot product of $\mathbf{PY2W2}$ and $\mathbf{CNTS}$ and divide by $n^2$

## Functions

In [1]:
import numpy as np
import scipy.sparse
import time

In [2]:
# for scipy sparse A
def var_est_log(n, p, y, A, z):    
    YW = y * A.dot(z-p)/(p*(1-p))

    P = np.power(np.ones(n)*p, z) + np.power(1 - np.ones(n)*p, 1 - z) - 1    # assignment probabilities
    lP = np.log(P)     # log of assignment probabilities

    PY2W2 = np.exp(A.dot(lP)) * YW**2 
    
    N = [np.nonzero(A[[i],:])[1] for i in range(n)]  # neighbors
  
    dep_neighbors = A.dot(A.transpose())
    M = [np.nonzero(dep_neighbors[[i],:])[1] for i in range(n)] # dependencies

    T1,T2 = 0,0
    for i in range(n):
        T1 += YW[i] * YW[M[i]].dot(1 - np.exp((A[[i],:]*A[M[i],:]).dot(lP)))
        T2 += PY2W2[i] * sum([2**len(N[j]) - 2**len(np.setdiff1d(N[j],N[i])) for j in M[i]])

    return (T1+T2)/n**2

# for scipy sparse A
def var_est_log2(n, p, y, A, z, N, M):    
    YW = y * A.dot(z-p)/(p*(1-p))

    P = np.power(np.ones(n)*p, z) + np.power(1 - np.ones(n)*p, 1 - z) - 1    # assignment probabilities
    lP = np.log(P)     # log of assignment probabilities

    PY2W2 = np.exp(A.dot(lP)) * YW**2 

    T1,T2 = 0,0
    for i in range(n):
        T1 += YW[i] * YW[M[i]].dot(1 - np.exp((A[[i],:]*A[M[i],:]).dot(lP)))
        T2 += PY2W2[i] * sum([2**len(N[j]) - 2**len(np.setdiff1d(N[j],N[i])) for j in M[i]])

    return (T1+T2)/n**2

In [3]:
# for scipy sparse A
def var_est_sp(n, p, y, A, z):
    '''
    n : int
        size of the population
    p : float
        treatment probability
    y : numpy array
        observations
    A : scipy SPARSE matrix 
        adjacency matrix where (i,j)th entry = 1 iff j's treatment affects i's outcome
    z : numpy array
        realized treatment assignment vector
    '''
    zz = z/p - (1-z)/(1-p)
    w = A.dot(zz)
    YW = y * w
    YW_sq = np.square(YW)

    V = np.zeros(n)
    PY2W2 = np.zeros(n)
    CNTS = np.zeros(n)

    prob_p = np.power(np.ones(n)*p, z) 
    prob_1_minus_p = np.power(1 - np.ones(n)*p, 1 - z)

    dep_neighbors = A.dot(A.transpose())
    
    for i in np.arange(n):
        Ni = np.nonzero(A[[i],:])[1]
        Mi = np.nonzero(dep_neighbors[[i],:])[1] # dependency neighbor's indices

        Pi = np.zeros(len(Mi))
        COVi = np.zeros(len(Mi))
        sum = 0
        for j in np.arange(len(Mi)):
            Nj = np.nonzero(A[[Mi[j]], :])[1]
            Ni_or_Nj = np.union1d(Ni,Nj)

            # Compute Pi
            mult_p = prob_p[Ni_or_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_or_Nj]
            Pi[j] = np.prod(mult_p) * np.prod(mult_1_minus_p)

            # Compute COVi
            Ni_and_Nj = np.intersect1d(Ni,Nj)
            mult_p = prob_p[Ni_and_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_and_Nj]
            temp = np.prod(mult_p) * np.prod(mult_1_minus_p)
            COVi[j] = Pi[j] * (1 - temp)

            # Compute CNTS[i]
            Nj_minus_Ni = np.setdiff1d(Nj,Ni)
            sum = sum + (2**len(Nj) - 2**len(Nj_minus_Ni))


        # Compute V
        V[i] = np.sum(1/Pi * YW[Mi] * COVi)

        # Compute PY2W2
        mult_p = prob_p[Ni]
        mult_1_minus_p = prob_1_minus_p[Ni]
        PY2W2[i] = np.prod(mult_p) * np.prod(mult_1_minus_p) * YW_sq[i]

        # Compute CNTS
        CNTS[i] = sum
    
    term1 = (1/n**2) * np.dot(YW, V)
    term2 = (1/n**2) * np.dot(PY2W2, CNTS)
    return term1+term2



In [4]:
# for numpy array A
def var_est_np(n, p, y, A, z):
    '''
    n : int
        size of the population
    p : float
        treatment probability
    y : numpy array
        observations
    A : numpy matrix 
        adjacency matrix where (i,j)th entry = 1 iff j's treatment affects i's outcome
    z : numpy array
        realized treatment assignment vector
    '''
    zz = z/p - (1-z)/(1-p)
    w = A.dot(zz)
    YW = y * w
    YW_sq = np.square(YW)

    V = np.zeros(n)
    PY2W2 = np.zeros(n)
    CNTS = np.zeros(n)

    prob_p = np.power(np.ones(n)*p, z) 
    prob_1_minus_p = np.power(1 - np.ones(n)*p, 1 - z)

    dep_neighbors = np.dot(A,A.T)
    
    for i in np.arange(n):
        Ni = np.nonzero(A[[i],:])[1]
        Mi = np.nonzero(dep_neighbors[[i],:])[1] # shares an in-neighbor
        
        Pi = np.zeros(len(Mi))
        COVi = np.zeros(len(Mi))
        sum = 0
        for j in np.arange(len(Mi)):
            Nj = np.nonzero(A[[Mi[j]], :])[1]
            Ni_or_Nj = np.union1d(Ni,Nj)

            # Compute Pi
            mult_p = prob_p[Ni_or_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_or_Nj]
            Pi[j] = np.prod(mult_p) * np.prod(mult_1_minus_p)

            # Compute COVi
            Ni_and_Nj = np.intersect1d(Ni,Nj)
            mult_p = prob_p[Ni_and_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_and_Nj]
            temp = np.prod(mult_p) * np.prod(mult_1_minus_p)
            COVi[j] = Pi[j] * (1 - temp)

            # Compute CNTS[i]
            Nj_minus_Ni = np.setdiff1d(Nj,Ni)
            sum = sum + (2**len(Nj) - 2**len(Nj_minus_Ni))


        # Compute V
        V[i] = np.sum(1/Pi * YW[Mi] * COVi)

        # Compute PY2W2
        mult_p = prob_p[Ni]
        mult_1_minus_p = prob_1_minus_p[Ni]
        PY2W2[i] = np.prod(mult_p) * np.prod(mult_1_minus_p) * YW_sq[i]

        # Compute CNTS
        CNTS[i] = sum
    
    #print("V: {}".format(V))
    #print("PY2W2: {}".format(PY2W2))
    #print("CNTS: {}\n".format(CNTS))
    term1 = (1/n**2) * np.dot(YW, V)
    #print("term1: {}".format(term1))
    term2 = (1/n**2) * np.dot(PY2W2, CNTS)
    #print("term2: {}".format(term2))
    return term1+term2

In [5]:
def var_bound(n, p, A, C, alp, beta=1):
    '''
    Returns the conservative upper bound on the variance of the SNIPE(beta) estimator

    n (int): size of the population
    p (float): treatment probability
    A (scipy sparse array): adjacency matrix
    C (scipy sparse array): weighted adjacency matrix
    alp (numpy array): baseline effects
    beta (int): degree of the potential outcomes model
    '''
    in_deg = scipy.sparse.diags(np.array(A.sum(axis=1)).flatten(),0)  # array of the in-degree of each node
    out_deg = scipy.sparse.diags(np.array(A.sum(axis=0)).flatten(),0)  # array of the out-degree of each node
    in_deg = in_deg.tocsr() 
    out_deg = out_deg.tocsr() 

    d_in = in_deg.max()
    d_out = out_deg.max()
    temp = max(4 * (beta**2), (1 / (p*(1-p))))

    if beta == 1:
        Ymax = np.amax(scipy.sparse.diags(np.array(C.sum(axis=1)).flatten(),0) + alp)
    else:
        Ymax = np.amax(1 + alp)

    bound = (1/n) * d_in * d_out * (Ymax**2) * (np.exp(1) * d_in * temp)**beta * (1/beta)**beta
    return bound

In [6]:
linear_pom = lambda C,alpha, z : C.dot(z) + alpha
bernoulli = lambda n,p : (np.random.rand(n) < p) + 0

def erdos_renyi(n,p):
    '''
    Generates a random network of n nodes using the Erdos-Renyi method,
    where the probability that an edge exists between two nodes is p.

    Returns the adjacency matrix of the network as an n by n scipy array
    '''
    A = np.random.rand(n,n)
    A = (A < p) + 0
    A[range(n),range(n)] = 1   # everyone is affected by their own treatment
    return scipy.sparse.csr_array(A)

# for scipy sparse A
def simpleWeights(A, diag=5, offdiag=5, rand_diag=np.array([]), rand_offdiag=np.array([])):
    '''
    Returns weights generated from simpler model

    A (scipy array): adjacency matrix of the network
    diag (float): maximum norm of direct effects
    offidiag (float): maximum norm of the indirect effects
    '''
    n = A.shape[0]

    if rand_offdiag.size == 0:
        rand_offdiag = np.random.rand(n)
    C_offdiag = offdiag*rand_offdiag

    in_deg = scipy.sparse.diags(np.array(A.sum(axis=1)).flatten(),0)  # array of the in-degree of each node
    C = in_deg.dot(A - scipy.sparse.eye(n))
    col_sum = np.array(C.sum(axis=0)).flatten()
    col_sum[col_sum==0] = 1
    temp = scipy.sparse.diags(C_offdiag/col_sum)
    C = C.dot(temp)

    if rand_diag.size == 0:
        rand_diag = np.random.rand(n)
    C_diag = diag*rand_diag
    C.setdiag(C_diag)

    return C

def est_us(n, p, y, A, z):
    '''
    Returns an estimate of the TTE using our proposed estimator

    n (int): number of individuals
    p (float): treatment probability
    y (numpy array): observations
    A (numpy or scipy array): network adjacency matrix
    z (numpy array): treatment vector
    '''
    zz = z/p - (1-z)/(1-p)
    return 1/n * y.dot(A.dot(zz))

# for numpy array A
def simpleWeights_np(A, diag=5, offdiag=5, rand_diag=np.array([]), rand_offdiag=np.array([])):
    '''
    Returns weights generated from simpler model

    A (numpy array): adjacency matrix of the network
    diag (float): maximum norm of direct effects
    offidiag (float): maximum norm of the indirect effects
    '''
    n = A.shape[0]

    if rand_offdiag.size == 0:
        rand_offdiag = np.random.rand(n)
    C_offdiag = offdiag*rand_offdiag

    in_deg = np.diag(np.array(A.sum(axis=1)).flatten(),0)  # array of the in-degree of each node
    C = in_deg.dot(A - np.eye(n))
    col_sum = np.array(C.sum(axis=0)).flatten()
    col_sum[col_sum==0] = 1
    temp = np.diag(C_offdiag/col_sum)
    C = C.dot(temp)

    if rand_diag.size == 0:
        rand_diag = np.random.rand(n)
    C_diag = diag*rand_diag
    C[range(n),range(n)] = C_diag

    return C

## Experiments

### Testing different versions of variance estimator implementation


In [10]:
n = 7500
r = 2
diag = 1
offdiag = r*diag
p = 0.2

# Create Weighted Adjcency matrix
deg = 10
A = erdos_renyi(n,deg/n)
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten() # baseline effects
C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

# Potential outcomes function
fy = lambda z: linear_pom(C,alpha,z)

TTE = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
print("Ground-Truth TTE: {}\n".format(TTE))

T = 10
TTE_hat, TTE_var_hat = np.zeros(T), np.zeros(T)

N = [np.nonzero(A[[i],:])[1] for i in range(n)]  # neighbors
dep_neighbors = A.dot(A.transpose())
M = [np.nonzero(dep_neighbors[[i],:])[1] for i in range(n)] # dependencies

time1 = time.time()
for i in range(T):
    z = bernoulli(n,p)
    y = fy(z)

    TTE_hat[i] = est_us(n, p, y, A, z)
    TTE_var_hat[i] = var_est_log2(n, p, y, A, z, N, M)
print("Loop took: {} minutes to run.".format((time.time()-time1)/60))

bound = var_bound(n, p, A, C, alpha)

exp_var = np.sum((TTE_hat-TTE)**2)/T
print("MSE (Experimental Variance): {}".format(exp_var))
print("Variance Bound: {}\n".format(bound))

print("Variance Estimate (new): {}".format(np.sum(TTE_var_hat)/T))
print("Variance Estimator bias: {}\n".format(((np.sum(TTE_var_hat)/T) - exp_var)))

Ground-Truth TTE: 1.5143968790448863

Loop took: 3.5189237793286643 minutes to run.
MSE (Experimental Variance): 0.14960705092819565
Variance Bound: 2562.4923139572734

Variance Estimate (new): 752.6014893873169
Variance Estimator bias: 752.4518823363886



In [7]:
n = 5000
r = 2
diag = 1
offdiag = r*diag
p = 0.2

# Create Weighted Adjcency matrix
deg = 10
A = erdos_renyi(n,deg/n)
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten() # baseline effects
C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

# Potential outcomes function
fy = lambda z: linear_pom(C,alpha,z)

TTE = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
print("Ground-Truth TTE1: {}\n".format(TTE))

T = 10
TTE_hat, TTE_var_hat1, TTE_var_hat2, TTE_var_hat3 = np.zeros(T), np.zeros(T), np.zeros(T), np.zeros(T)

Ground-Truth TTE1: 1.5163220983889918



/Users/mayleencortez/Desktop/NetworkCausalInference/ClusterRD/low-order-clusterRD/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
N = [np.nonzero(A[[i],:])[1] for i in range(n)]  # neighbors
dep_neighbors = A.dot(A.transpose())
M = [np.nonzero(dep_neighbors[[i],:])[1] for i in range(n)] # dependencies

time1 = time.time()
for i in range(T):
    z = bernoulli(n,p)
    y = fy(z)

    TTE_hat[i] = est_us(n, p, y, A, z)
    TTE_var_hat2[i] = var_est_log(n, p, y, A, z)
    TTE_var_hat3[i] = var_est_log2(n, p, y, A, z, N, M)

bound = var_bound(n, p, A, C, alpha)

exp_var = np.sum((TTE_hat-TTE)**2)/T
print("MSE (Experimental Variance): {}".format(exp_var))
print("Variance Bound: {}\n".format(bound))

print("Variance Estimate (new): {}".format(np.sum(TTE_var_hat2)/T))
print("Variance Estimator bias: {}\n".format(((np.sum(TTE_var_hat2)/T) - exp_var)))

print("Variance Estimate (with adjustment): {}".format(np.sum(TTE_var_hat3)/T))
print("Variance Estimator bias: {}\n".format(((np.sum(TTE_var_hat3)/T) - exp_var)))

print("Total time (minutes): {}".format((time.time()-time1)/60))

MSE (Experimental Variance): 0.1813056782212066
Variance Bound: 1775.742722168508

Variance Estimate (new): 413.26817825792415
Variance Estimator bias: 413.08687257970297

Variance Estimate (with adjustment): 413.26817825792415
Variance Estimator bias: 413.08687257970297

Total time (minutes): 4.798470648129781


In [16]:
n = 5000
r = 2
diag = 1
offdiag = r*diag
p = 0.2

# Create Weighted Adjcency matrix
deg = 10
A = erdos_renyi(n,deg/n)
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten() # baseline effects
C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

# Potential outcomes function
fy = lambda z: linear_pom(C,alpha,z)

TTE = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
print("Ground-Truth TTE1: {}\n".format(TTE))

T = 10
TTE_hat, TTE_var_hat1, TTE_var_hat2, TTE_var_hat3 = np.zeros(T), np.zeros(T), np.zeros(T), np.zeros(T)

time1 = time.time()
for i in range(T):
    z = bernoulli(n,p)
    y = fy(z)

    TTE_hat[i] = est_us(n, p, y, A, z)
    TTE_var_hat1[i] = var_est_sp(n, p, y, A, z)
print("My original code took: {} minutes to run.".format((time.time()-time1)/60))

time1 = time.time()
for i in range(T):
    z = bernoulli(n,p)
    y = fy(z)

    TTE_hat[i] = est_us(n, p, y, A, z)
    TTE_var_hat2[i] = var_est_log(n, p, y, A, z)
print("Matt's new code took: {} minutes to run.".format((time.time()-time1)/60))

time1 = time.time()
N = [np.nonzero(A[[i],:])[1] for i in range(n)]  # neighbors
dep_neighbors = A.dot(A.transpose())
M = [np.nonzero(dep_neighbors[[i],:])[1] for i in range(n)] # dependencies
for i in range(T):
    z = bernoulli(n,p)
    y = fy(z)

    TTE_hat[i] = est_us(n, p, y, A, z)
    TTE_var_hat3[i] = var_est_log2(n, p, y, A, z, N, M)
print("Matt's new code with adjustment took: {} minutes to run.".format((time.time()-time1)/60))

exp_var = np.sum((TTE_hat-TTE)**2)/T
print("Variance Estimate (my original): {}".format(np.sum(TTE_var_hat1)/T))
print("Variance Estimate (Matt's): {}".format(np.sum(TTE_var_hat2)/T))
print("Variance Estimate (Matt's with adjustment): {}".format(np.sum(TTE_var_hat3)/T))

Ground-Truth TTE1: 1.4930987946410959

My original code took: 8.69225392738978 minutes to run.
Matt's new code took: 2.4298123637835185 minutes to run.
Matt's new code with adjustment took: 2.35362042983373 minutes to run.
Variance Estimate (my original): 447.2034749808164
Variance Estimate (Matt's): 453.67712260747277
Variance Estimate (Matt's with adjustment): 490.1557171090725


### Regular experiment

In [6]:
n = 5000
r = 2
diag = 1
offdiag = r*diag
p = 0.2

# Create Weighted Adjcency matrix
deg = 10
A = erdos_renyi(n,deg/n)
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten() # baseline effects
C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

# Potential outcomes function
fy = lambda z: linear_pom(C,alpha,z)

TTE = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
print("Ground-Truth TTE1: {}\n".format(TTE))

T = 100
TTE_hat, TTE_var_hat_sp = np.zeros(T), np.zeros(T)

Ground-Truth TTE1: 1.5311264185044857



/Users/mayleencortez/Desktop/NetworkCausalInference/ClusterRD/low-order-clusterRD/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [7]:
time1 = time.time()

for i in range(T):
    time2 = time.time()
    z = bernoulli(n,p)
    y = fy(z)

    TTE_hat[i] = est_us(n, p, y, A, z)
    TTE_var_hat_sp[i] = var_est_sp(n, p, y, A, z)

bound = var_bound(n, p, A, C, alpha)

print("SNIPE: {}".format(np.sum(TTE_hat)/T))
print("SNIPE bias: {}\n".format(((np.sum(TTE_hat)/T) - TTE)/TTE))

exp_var = np.sum((TTE_hat-TTE)**2)/T
print("MSE (Experimental Variance): {}".format(exp_var))
print("Variance Bound: {}\n".format(bound))

print("Variance Estimate (sp): {}".format(np.sum(TTE_var_hat_sp)/T))
print("Variance Estimator bias: {}\n".format(((np.sum(TTE_var_hat_sp)/T) - exp_var)))

print("Total time (minutes): {}".format((time.time()-time1)/60))


SNIPE: 1.541482789564857
SNIPE bias: 0.006763890254396397

MSE (Experimental Variance): 0.18657603480817006
Variance Bound: 1249.6571388475677

Variance Estimate (sp): 308.5380731814132
Variance Estimator bias: 308.351497146605

Total time (minutes): 87.46593656539918


### Testing np.dot and A.dot

In [8]:
n = 10
r = 2
diag = 1
offdiag = r*diag
p = 0.2

# Create Weighted Adjcency matrix
deg = 5
Asp = erdos_renyi(n,deg/n)
Anp = Asp.toarray()
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten() # baseline effects
Csp = simpleWeights(Asp, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())
Cnp = simpleWeights_np(Anp, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

# Potential outcomes function
fy = lambda z: linear_pom(Csp,alpha,z)
fy2 = lambda z: linear_pom(Cnp,alpha,z)

TTEsp = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
print("Ground-Truth TTEsp: {}\n".format(TTEsp))

TTEnp = 1/n * np.sum((fy2(np.ones(n)) - fy2(np.zeros(n))))
print("Ground-Truth TTEnp: {}\n".format(TTEnp))

# Experiment
z = bernoulli(n,p)
y = fy(z)

Ground-Truth TTEsp: 1.6934272551775629

Ground-Truth TTEnp: 1.6934272551775629



In [9]:
zz = z/p - (1-z)/(1-p)
w_sp = Asp.dot(zz)
w_np = Anp.dot(zz)
print(w_sp ==  w_np)
print('\n')

Mi_sp_npdot = np.dot(Asp,Asp.T)
Mi_sp_spdot = Asp.dot(Asp.transpose())

Mi_np_npdot = np.dot(Anp,Anp.T)
Mi_np_spdot = Anp.dot(Anp.transpose())

print("np.dot and A.dot are the same thing on a scipy sparse matrix: {}".format(np.array_equal(Mi_sp_npdot.toarray(),Mi_sp_spdot.toarray())))
print("np.dot and A.dot are the same thing on a numpy matrix: {}".format(np.array_equal(Mi_np_npdot,Mi_np_spdot)))
print("np.dot on dense matrix and A.dot on scipy sparse matrix are the same: {}\n".format(Mi_sp_spdot == Mi_np_npdot))

[ True  True  True  True  True  True  True  True  True  True]


np.dot and A.dot are the same thing on a scipy sparse matrix: False
np.dot and A.dot are the same thing on a numpy matrix: True
np.dot on dense matrix and A.dot on scipy sparse matrix are the same: [[ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]]



### Beta greater than 1

In [10]:
from scipy import special
SNIPE_beta = lambda n,y,w : np.sum(y*w)/n

def SNIPE_weights(n, p, A, z, beta):
  treated_neighb = A.dot(z)
  control_neighb = A.dot(1-z)
  W = np.zeros(n)
  for i in range(n):
    w = 0
    a_lim = min(beta,int(treated_neighb[i]))
    for a in range(a_lim+1):
      b_lim = min(beta - a,int(control_neighb[i]))
      for b in range(b_lim+1):
        w = w + ((1-p)**(a+b) - (-p)**(a+b)) * p**(-a) * (p-1)**(-b) * special.binom(treated_neighb[i],a)  * special.binom(control_neighb[i],b)
    W[i] = w

  return W

SNIPE_beta = lambda n,y,w : np.sum(y*w)/n

def SNIPE_beta_OG(n, p, y, A, z, beta):
  # n = z.size
  # z = z.reshape((n,1))
  treated_neighb = A.dot(z)
  control_neighb = A.dot(1-z)
  est = 0
  for i in range(n):
    w = 0
    a_lim = min(beta,int(treated_neighb[i]))
    for a in range(a_lim+1):
      b_lim = min(beta - a,int(control_neighb[i]))
      for b in range(b_lim+1):
        w = w + ((1-p)**(a+b) - (-p)**(a+b)) * p**(-a) * (p-1)**(-b) * special.binom(treated_neighb[i],a)  * special.binom(control_neighb[i],b)
    est = est + y[i]*w

  return est/n

# Scale down the effects of higher order terms
a1 = 1      # for linear effects
a2 = 1    # for quadratic effects
a3 = 1   # for cubic effects
a4 = 1   # for quartic effects

# Define f(z)
f_linear = lambda alpha, z, gz: alpha + a1*z
f_quadratic = lambda alpha, z, gz: alpha + a1*z + a2*np.multiply(gz,gz)
f_cubic = lambda alpha, z, gz: alpha + a1*z + a2*np.multiply(gz,gz) + a3*np.power(gz,3)
f_quartic = lambda alpha, z, gz: alpha + a1*z + a2*np.multiply(gz,gz) + a3*np.power(gz,3) + a4*np.power(gz,4)

def ppom(beta, C, alpha):
  '''
  Returns k-degree polynomial potential outcomes function fy
  
  f (function): must be of the form f(z) = alpha + z + a2*z^2 + a3*z^3 + ... + ak*z^k
  C (np.array): weighted adjacency matrix
  alpha (np.array): vector of null effects
  '''
  # n = C.shape[0]
  # assert np.all(f(alpha, np.zeros(n), np.zeros(n)) == alpha), 'f(0) should equal alpha'
  #assert np.all(np.around(f(alpha, np.ones(n)) - alpha - np.ones(n), 10) >= 0), 'f must include linear component'

  if beta == 0:
      return lambda z: alpha + a1*z
  elif beta == 1:
      f = f_linear
      return lambda z: alpha + a1*C.dot(z)
  else:
      g = lambda z : C.dot(z) / np.array(np.sum(C,1)).flatten()
      if beta == 2:
          f = f_quadratic
      elif beta == 3:
          f = f_cubic
      elif beta == 4:
          f = f_quartic
      else:
          print("ERROR: invalid degree")
      return lambda z: f(alpha, C.dot(z), g(z)) 

In [11]:
n = 15000
r = 2
diag = 1
offdiag = r*diag
p = 0.2
beta = 2

# Create Weighted Adjcency matrix
deg = 10
A, A_np = erdos_renyi(n,deg/n)
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten() # baseline effects
C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

# Potential outcomes function
fy = ppom(beta, C, alpha)

TTE = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
print("Ground-Truth TTE: {}\n".format(TTE))

ValueError: too many values to unpack (expected 2)